# VannaAI

Uso de vanna ia para la bd minitest_sqlite3

In [2]:
import sqlite3
import pandas as pd
from vanna.remote import VannaDefault

from dotenv import load_dotenv
import os


In [ ]:

load_dotenv(dotenv_path='../.env')

# https://vanna.ai/account/profile

api_key = os.getenv('RAPIDAPI_KEY')
vanna_model_name = os.getenv('RAPIDAPI_HOST')

vn = VannaDefault(model=vanna_model_name, api_key=api_key)
conn = sqlite3.connect('Dbs/db_minitest.sqlite3')


- DDL: definición de las tablas.


In [ ]:
vn.train(
    ddl="""
    CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    age INTEGER,
    email VARCHAR(100),
    created_at DATE
);
    CREATE TABLE profiles (
    user_id INTEGER PRIMARY KEY,
    address VARCHAR(255),
    phone VARCHAR(50),
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product VARCHAR(100),
    amount FLOAT,
    order_date DATE,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);


"""
)vn.train(
    ddl="""
    CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    age INTEGER,
    email VARCHAR(100),
    created_at DATE
);
    CREATE TABLE profiles (
    user_id INTEGER PRIMARY KEY,
    address VARCHAR(255),
    phone VARCHAR(50),
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product VARCHAR(100),
    amount FLOAT,
    order_date DATE,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);


"""
)

Adding ddl: 
    CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    age INTEGER,
    email VARCHAR(100),
    created_at DATE
);
    CREATE TABLE profiles (
    user_id INTEGER PRIMARY KEY,
    address VARCHAR(255),
    phone VARCHAR(50),
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product VARCHAR(100),
    amount FLOAT,
    order_date DATE,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);





'16887736-ddl'

In [5]:
vn.train(
    documentation="El usuario puede usar el nombre de las personas en lugar del pedido."
)

Adding documentation....


'3442628-doc'

In [6]:
vn.train(
    documentation="""Términos comunes del negocio:
- "contacto del usuario" = nombre, email, teléfono, dirección
- "usuarios recientes" = registrados este año (`created_at`)
- "pedidos recientes" = por `order_date`
- "historial de compras" = suma de `amount` en `orders`
- "ciudades activas" = direcciones de usuarios con pedidos
- Grupos de edad:
  - Jóvenes: menores de 25
  - Adultos: entre 25 y 40
  - Seniors: mayores de 40"""
)

Adding documentation....


'3442629-doc'

In [7]:
vn.train(
    question="¿Cuáles son los primeros 5 contactos de usuarios mayores de 30 años que se registraron este año?",
    sql="""
    SELECT u.name, u.email, p.phone, p.address, u.created_at
    FROM users u
    JOIN profiles p ON u.user_id = p.user_id
    WHERE u.age > 30
      AND strftime('%Y', u.created_at) = strftime('%Y', DATE('now'))
    ORDER BY u.created_at ASC
    LIMIT 5;
    """
)


'856b141ad012125bf8c3c72517ef4069-sql'

In [8]:
vn.train(
    question="Muéstrame todos los datos de los usuarios con sus pedidos más recientes, ordenados cronológicamente.",
    sql="""
    SELECT u.*, p.phone, p.address, o.*
    FROM users u
    JOIN profiles p ON u.user_id = p.user_id
    JOIN orders o ON u.user_id = o.user_id
    ORDER BY o.order_date DESC;
    """
)

'61f4a2e4aa4571e1e86a0326e27ffe40-sql'

In [9]:
vn.train(
    question="¿En qué ciudades viven usuarios que han hecho al menos dos pedidos?",
    sql="""
    SELECT DISTINCT p.address
    FROM users u
    JOIN profiles p ON u.user_id = p.user_id
    JOIN orders o ON u.user_id = o.user_id
    GROUP BY u.user_id
    HAVING COUNT(o.order_id) >= 2;
    """
)


'3840e835343b7d806d48f578b4906584-sql'

In [22]:
vn.train(
    question="¿Cuánto ha gastado cada cliente que ha superado los 700 dólares en pedidos?",
    sql="""
    SELECT u.name, u.email, SUM(o.amount) AS total_gastado
    FROM users u
    JOIN orders o ON u.user_id = o.user_id
    GROUP BY u.user_id
    HAVING total_gastado > 700;
    """
)


'3cc92501e645f0b467a9957a4fecbf35-sql'

In [20]:
vn.train(
    question="¿Cuántos usuarios hay en cada grupo de edad: jóvenes, adultos y seniors?",
    sql="""
    SELECT 
      CASE 
        WHEN age < 25 THEN 'Jóvenes'
        WHEN age BETWEEN 25 AND 40 THEN 'Adultos'
        ELSE 'Seniors'
      END AS grupo_edad,
      COUNT(*) AS total
    FROM users
    GROUP BY grupo_edad;
    """
)


'405756c8a6326646ae5ea9c3b110e403-sql'

In [21]:
vn.train(
    question="¿Cuántos usuarios hay en cada grupo de edad",
    sql="""
    SELECT 
      CASE 
        WHEN age < 25 THEN 'Jóvenes'
        WHEN age BETWEEN 25 AND 40 THEN 'Adultos'
        ELSE 'Seniors'
      END AS grupo_edad,
      COUNT(*) AS total
    FROM users
    GROUP BY grupo_edad;
    """
)


'31d6f34f6b3b44b88cf4f687d1204730-sql'

Para visualizar lo almacenado hasta el momento:

In [23]:
vn.get_training_data()

,id,training_data_type,question,content
0,1227013-sql,sql,¿En qué ciudades viven usuarios que han hecho ...,\n SELECT DISTINCT p.address\n FROM user...
1,3442628-doc,documentation,None,El usuario puede usar el nombre de las persona...
2,1227014-sql,sql,¿Cuántos usuarios hay en cada grupo de edad: j...,\n SELECT \n CASE \n WHEN age <...
3,1227012-sql,sql,Muéstrame todos los datos de los usuarios con ...,"\n SELECT u.*, p.phone, p.address, o.*\n ..."
4,1227016-sql,sql,¿Cuánto ha gastado cada cliente que ha superad...,"\n SELECT u.name, u.email, SUM(o.amount) AS..."
5,16887736-ddl,ddl,None,\n CREATE TABLE users (\n user_id INTEGE...
6,1227015-sql,sql,¿Cuántos usuarios hay en cada grupo de edad,\n SELECT \n CASE \n WHEN age <...
7,1227011-sql,sql,¿Cuáles son los primeros 5 contactos de usuari...,"\n SELECT u.name, u.email, p.phone, p.addre..."
8,3442629-doc,documentation,None,"Términos comunes del negocio:\n- ""contacto del..."


In [18]:
vn.remove_training_data('3436311-doc') # se ingresa el id que se visualiza en la tabla anterior

True

### GEN SQL

In [24]:
query = vn.generate_sql(
    "cual fue la ultima orden pedida?"
)

SQL Prompt: [{'role': 'system', 'content': 'You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE users (\n    user_id INTEGER PRIMARY KEY,\n    name VARCHAR(100),\n    age INTEGER,\n    email VARCHAR(100),\n    created_at DATE\n);\n    CREATE TABLE profiles (\n    user_id INTEGER PRIMARY KEY,\n    address VARCHAR(255),\n    phone VARCHAR(50),\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\nCREATE TABLE orders (\n    order_id INTEGER PRIMARY KEY,\n    user_id INTEGER,\n    product VARCHAR(100),\n    amount FLOAT,\n    order_date DATE,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\n\n\n\n===Additional Context \n\nEl usuario puede usar el nombre de las personas en lugar del pedido.\n\nTérminos comunes del negocio:\n- "contacto del usuario" = nombre, email, teléfono, dirección\n- "usuarios 

In [ ]:
query1 = vn.generate_sql("en que ciudades viven personas que hayan hecho al menos dos pedidos, tambien quiero saber la cantidad de pedidos de estas personas")
query2 = vn.generate_sql("cual es el total gastado por cada usuario que haya gastado mas de 700 dolares en pedidos?")
query_usuarios_recientes = vn.generate_sql("contactos de usuarios mayores de 30 años registrados este año, ordenados por fecha de registro, primeros 5")
query_ciudades_activas = vn.generate_sql("ubicaciones únicas donde viven usuarios que hayan realizado al menos dos pedidos")
query_segmentacion_demografica = vn.generate_sql("desglose del número de usuarios por grupo de edad: jóvenes, adultos y seniors")
query_informe_combinado = vn.generate_sql("datos de contacto e información personal de cada usuario junto con sus pedidos más recientes, ordenados cronológicamente")

In [34]:
df_ultima_orden = pd.read_sql_query(query, conn)
df_ciudades_pedidos = pd.read_sql_query(query1, conn)
df_total_gastado = pd.read_sql_query(query2, conn)
df_usuarios_recientes = pd.read_sql_query(query_usuarios_recientes, conn)
df_ciudades_activas = pd.read_sql_query(query_ciudades_activas, conn)
df_segmentacion_demografica = pd.read_sql_query(query_segmentacion_demografica, conn)
df_informe_combinado = pd.read_sql_query(query_informe_combinado, conn)

In [35]:
print("df_ultima_orden:")
print(df_ultima_orden)
print("\ndf_ciudades_pedidos:")
print(df_ciudades_pedidos)
print("\ndf_total_gastado:")
print(df_total_gastado)
print("\ndf_usuarios_recientes:")
print(df_usuarios_recientes)
print("\ndf_ciudades_activas:")
print(df_ciudades_activas)
print("\ndf_segmentacion_demografica:")
print(df_segmentacion_demografica)
print("\ndf_informe_combinado:")
print(df_informe_combinado)

df_ultima_orden:
   order_id  user_id  product  amount  order_date
0      1015        8  Charger    20.0  2025-06-06

df_ciudades_pedidos:
         address  cantidad_pedidos
0    123 Main St                 2
1    321 Pine St                 3
2  654 Cedar Ave                 2
3   987 Birch Rd                 3
4  135 Willow Dr                 2

df_total_gastado:
    name  total_gastado
0  Alice         1225.5

df_usuarios_recientes:
     name               email     phone        address  created_at
0   Diana   diana@example.com  555-3456    321 Pine St  2025-03-15
1  Edward  edward@example.com  555-7890  654 Cedar Ave  2025-04-10
2  Hannah  hannah@example.com  555-0123  135 Willow Dr  2025-06-01

df_ciudades_activas:
         address
0    123 Main St
1    321 Pine St
2  654 Cedar Ave
3   987 Birch Rd
4  135 Willow Dr

df_segmentacion_demografica:
  grupo_edad  total
0    Adultos      3
1    Jóvenes      2
2    Seniors      3

df_informe_combinado:
    user_id     name  age          

##### Preguntas extra

In [36]:
# Generar dos nuevas preguntas de negocio y obtener el SQL correspondiente
query_top_productos = vn.generate_sql("¿Cuáles son los 3 productos más vendidos y cuántas veces se vendió cada uno?")
query_promedio_gasto = vn.generate_sql("¿Cuál es el gasto promedio por pedido para cada usuario?")

# Ejecutar las consultas y guardar los resultados en dataframes
df_top_productos = pd.read_sql_query(query_top_productos, conn)
df_promedio_gasto = pd.read_sql_query(query_promedio_gasto, conn)

# Imprimir los resultados
print("Top 3 productos más vendidos:")
print(df_top_productos)
print("\nGasto promedio por pedido para cada usuario:")
print(df_promedio_gasto)

SQL Prompt: [{'role': 'system', 'content': 'You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE users (\n    user_id INTEGER PRIMARY KEY,\n    name VARCHAR(100),\n    age INTEGER,\n    email VARCHAR(100),\n    created_at DATE\n);\n    CREATE TABLE profiles (\n    user_id INTEGER PRIMARY KEY,\n    address VARCHAR(255),\n    phone VARCHAR(50),\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\nCREATE TABLE orders (\n    order_id INTEGER PRIMARY KEY,\n    user_id INTEGER,\n    product VARCHAR(100),\n    amount FLOAT,\n    order_date DATE,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\n\n\n\n===Additional Context \n\nTérminos comunes del negocio:\n- "contacto del usuario" = nombre, email, teléfono, dirección\n- "usuarios recientes" = registrados este año (`created_at`)\n- "pedidos recientes" 